In [27]:

!pip install python-dotenv


In [28]:
import os
from dotenv import load_dotenv
import json
import base64
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

def authenticate():
    #Load .env
    load_dotenv()
    #DLAI Custom Key
    return "DLAI_CREDENTIALS", "DLAI_PROJECT", "gs://gcp-sc2-rlhf"

    #Decode key and store in .JSON
    SERVICE_ACCOUNT_KEY_STRING_B64 = os.getenv('SERVICE_ACCOUNT_KEY')
    SERVICE_ACCOUNT_KEY_BYTES_B64 = SERVICE_ACCOUNT_KEY_STRING_B64.encode("ascii")
    SERVICE_ACCOUNT_KEY_STRING_BYTES = base64.b64decode(SERVICE_ACCOUNT_KEY_BYTES_B64)
    SERVICE_ACCOUNT_KEY_STRING = SERVICE_ACCOUNT_KEY_STRING_BYTES.decode("ascii")

    SERVICE_ACCOUNT_KEY = json.loads(SERVICE_ACCOUNT_KEY_STRING)


    # Create credentials based on key from service account
    # Make sure your account has the roles listed in the Google Cloud Setup section
    credentials = Credentials.from_service_account_info(
        SERVICE_ACCOUNT_KEY,
        scopes=['https://www.googleapis.com/auth/cloud-platform'])

    if credentials.expired:
        credentials.refresh(Request())

    #Set project ID according to environment variable
    PROJECT_ID = os.getenv('PROJECT_ID')
    STAGING_BUCKET = os.getenv('STAGING_BUCKET')# 'gs://gcp-sc2-rlhf-staging'

    return credentials, PROJECT_ID, STAGING_BUCKET


In [1]:
!pip3 install google-cloud-pipeline-components
!pip3 install kfp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.8/441.8 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 9.3 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-2.7.0-py3-none-any.whl size=610411 sha256=9822391877530453247fdf2f53b82ab0632a64d3efc19c69033f5b184c7c5a3b
  Stored in directory: /root/.cache/pip/wheels/9e/7d/a4/f9d013e82681c9746ef10de3b00456163577a99279c5ed673d
  Created wheel for kfp-server-

In [2]:
# Import (RLFH is currently in preview)
from google_cloud_pipeline_components.preview.llm \
import rlhf_pipeline

In [3]:
# Import from KubeFlow pipelines
from kfp import compiler

In [4]:
# Define a path to the yaml file
RLHF_PIPELINE_PKG_PATH = "rlhf_pipeline.yaml"

In [5]:
# Execute the compile function
compiler.Compiler().compile(
    pipeline_func=rlhf_pipeline,
    package_path=RLHF_PIPELINE_PKG_PATH
)

In [6]:
# Print the first lines of the YAML file
!head rlhf_pipeline.yaml

# PIPELINE DEFINITION
# Name: rlhf-train-template
# Description: Performs reinforcement learning from human feedback.
# Inputs:
#    accelerator_type: str [Default: 'GPU']
#    deploy_model: bool [Default: True]
#    encryption_spec_key_name: str [Default: '']
#    eval_dataset: str
#    instruction: str
#    kl_coeff: float [Default: 0.1]


In [7]:
!cat rlhf_pipeline.yaml

# PIPELINE DEFINITION
# Name: rlhf-train-template
# Description: Performs reinforcement learning from human feedback.
# Inputs:
#    accelerator_type: str [Default: 'GPU']
#    deploy_model: bool [Default: True]
#    encryption_spec_key_name: str [Default: '']
#    eval_dataset: str
#    instruction: str
#    kl_coeff: float [Default: 0.1]
#    large_model_reference: str
#    location: str [Default: '{{$.pipeline_google_cloud_location}}']
#    model_display_name: str
#    preference_dataset: str
#    project: str [Default: '{{$.pipeline_google_cloud_project_id}}']
#    prompt_dataset: str
#    prompt_sequence_length: int [Default: 512.0]
#    reinforcement_learning_rate_multiplier: float [Default: 1.0]
#    reinforcement_learning_train_steps: int [Default: 1000.0]
#    reward_model_learning_rate_multiplier: float [Default: 1.0]
#    reward_model_train_steps: int [Default: 1000.0]
#    target_sequence_length: int [Default: 64.0]
#    tensorboard_resource_id: str [Default: '']
# Outputs:

In [10]:
# Preference dataset size
PREF_DATASET_SIZE = 3000

In [11]:
# Batch size is fixed at 64
BATCH_SIZE = 64

In [12]:
import math

In [13]:
REWARD_STEPS_PER_EPOCH = math.ceil(PREF_DATASET_SIZE / BATCH_SIZE)
print(REWARD_STEPS_PER_EPOCH)

47


In [14]:
REWARD_NUM_EPOCHS = 30

In [15]:
# Calculate number of steps in the reward model training
reward_model_train_steps = REWARD_STEPS_PER_EPOCH * REWARD_NUM_EPOCHS

In [16]:
print(reward_model_train_steps)

1410


# Number of reinforcement learning training steps

In [17]:
# Prompt dataset size
PROMPT_DATASET_SIZE = 2000

In [18]:
# Batch size is fixed at 64
BATCH_SIZE = 64

In [19]:
import math
RL_STEPS_PER_EPOCH = math.ceil(PROMPT_DATASET_SIZE / BATCH_SIZE)
print(RL_STEPS_PER_EPOCH)

32


In [20]:
RL_NUM_EPOCHS = 10

In [21]:
# Calculate the number of steps in the RL training
reinforcement_learning_train_steps = RL_STEPS_PER_EPOCH * RL_NUM_EPOCHS

In [22]:
print(reinforcement_learning_train_steps)

320


In [9]:
# Completed values for the dictionary
parameter_values={
        "preference_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/summarize_from_feedback_tfds/comparisons/train/*.jsonl",
        "prompt_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/reddit_tfds/train/*.jsonl",
        "eval_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text_small/reddit_tfds/val/*.jsonl",
        "large_model_reference": "llama-2-7b",
        "reward_model_train_steps": 1410,
        "reinforcement_learning_train_steps": 320, # results from the calculations above
        "reward_model_learning_rate_multiplier": 1.0,
        "reinforcement_learning_rate_multiplier": 1.0,
        "kl_coeff": 0.1, # increased to reduce reward hacking
        "instruction":\
    "Summarize in less than 50 words"}

In [23]:
parameter_values={
        "preference_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/summarize_from_feedback_tfds/comparisons/train/*.jsonl",
        "prompt_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/reddit_tfds/train/*.jsonl",
        "eval_dataset": \
    "gs://vertex-ai/generative-ai/rlhf/text/reddit_tfds/val/*.jsonl",
        "large_model_reference": "llama-2-7b",
        "reward_model_train_steps": 10000,
        "reinforcement_learning_train_steps": 10000,
        "reward_model_learning_rate_multiplier": 1.0,
        "reinforcement_learning_rate_multiplier": 0.2,
        "kl_coeff": 0.1,
        "instruction":\
    "Summarize in less than 50 words"}

In [30]:
# Authenticate in utils

credentials, PROJECT_ID, STAGING_BUCKET = authenticate()

# RLFH pipeline is available in this region
REGION = "europe-west4"

## Run the pipeline job on Vertex AI

In [31]:
import google.cloud.aiplatform as aiplatform

In [32]:
aiplatform.init(project = PROJECT_ID,
                location = REGION,
                credentials = credentials)

In [33]:
# Look at the path for the YAML file
RLHF_PIPELINE_PKG_PATH

'rlhf_pipeline.yaml'

In [36]:
job = aiplatform.PipelineJob(
    display_name="tutorial-rlhf-tuning",
    pipeline_root=STAGING_BUCKET,
    template_path=RLHF_PIPELINE_PKG_PATH,
    parameter_values=parameter_values)

In [ ]:
job.run()